In [12]:
import coffea
from git import Repo
import awkward as ak
import uproot
import dask_awkward as dak
mod = "Prayag Yadav"
local_repo = Repo(path='coffea')
local_branch = local_repo.active_branch.name
print("_______________________________________")
print("\tCurrent Configuration")
print("---------------------------------------")
print("Coffea Version: ", coffea.__version__)
print("Branch: \t", local_branch)
print("Modified by: \t", mod)
print("_______________________________________")


_______________________________________
	Current Configuration
---------------------------------------
Coffea Version:  0.1.dev3583+ge06c4b8
Branch: 	 master
Modified by: 	 Prayag Yadav
_______________________________________


In [68]:
from coffea.nanoevents import NanoEventsFactory, FCCSchema, FCC
test_file = '../../../coffea-fcc-analyses/data/p8_ee_ZH_ecm240/events_082532938.root'
file = uproot.open(test_file)

events = NanoEventsFactory.from_root(
    test_file+":events",
    entry_stop=100,
    schemaclass= FCC.get_schema(version="latest"),
    delayed = False,
    metadata=file["metadata"].arrays()
).events()

file.close()

In [69]:
events.metadata.m_names

<Array [['Electron', 'Muon', ..., 'Jet', 'EFlowTrack']] type='1 * var * string'>

In [3]:
events.fields

['AllMuonidx0',
 'EFlowNeutralHadron',
 'EFlowNeutralHadron_0',
 'EFlowNeutralHadron_1',
 'EFlowNeutralHadron_2',
 'EFlowNeutralHadronidx0',
 'EFlowNeutralHadronidx1',
 'EFlowNeutralHadronidx2',
 'EFlowPhoton',
 'EFlowPhoton_0',
 'EFlowPhoton_1',
 'EFlowPhoton_2',
 'EFlowPhotonidx0',
 'EFlowPhotonidx1',
 'EFlowPhotonidx2',
 'EFlowTrack',
 'EFlowTrack_0',
 'EFlowTrack_1',
 'EFlowTrackidx0',
 'EFlowTrackidx1',
 'Electronidx0',
 'Jet',
 'Jetidx0',
 'Jetidx1',
 'Jetidx2',
 'Jetidx3',
 'Jetidx4',
 'Jetidx5',
 'MCRecoAssociations',
 'MCRecoAssociationsidx0',
 'MCRecoAssociationsidx1',
 'MissingET',
 'MissingETidx0',
 'MissingETidx1',
 'MissingETidx2',
 'MissingETidx3',
 'MissingETidx4',
 'MissingETidx5',
 'Muonidx0',
 'Particle',
 'ParticleIDs',
 'ParticleIDs_0',
 'Particleidx0',
 'Particleidx1',
 'Photonidx0',
 'ReconstructedParticles',
 'ReconstructedParticlesidx0',
 'ReconstructedParticlesidx1',
 'ReconstructedParticlesidx2',
 'ReconstructedParticlesidx3',
 'ReconstructedParticlesidx4',
 

In [4]:
events.show

<bound method Array.show of <NanoEventsArray [FCC Events, FCC Events, ..., FCC Events] type='100 * event'>>

In [14]:
file = uproot.open(test_file)

In [16]:
file.classnames()

{'events;33': 'TTree',
 'events;32': 'TTree',
 'metadata;1': 'TTree',
 'run_metadata;1': 'TTree',
 'evt_metadata;1': 'TTree',
 'col_metadata;1': 'TTree'}

In [19]:
metadata = file["metadata"]
metadata.show()

name                 | typename                 | interpretation                
---------------------+--------------------------+-------------------------------
CollectionIDs        | podio::CollectionIDTable | AsGroup(<TBranchElement 'Co...
CollectionIDs/m_c... | std::vector<int32_t>     | AsJagged(AsDtype('>i4'), he...
CollectionIDs/m_n... | std::vector<std::string> | AsObjects(AsVector(True, As...


In [25]:
metadata['CollectionIDs'].show()

name                 | typename                 | interpretation                
---------------------+--------------------------+-------------------------------
CollectionIDs        | podio::CollectionIDTable | AsGroup(<TBranchElement 'Colle
m_collectionIDs      | std::vector<int32_t>     | AsJagged(AsDtype('>i4'), he...
m_names              | std::vector<std::string> | AsObjects(AsVector(True, As...


In [32]:
m = metadata.arrays()
m

<Array [{m_collectionIDs: [...], ...}] type='1 * {m_collectionIDs: var * in...'>

In [37]:
print(m.m_collectionIDs)

[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]]


In [40]:
file["col_metadata"].show()

name                 | typename                 | interpretation                
---------------------+--------------------------+-------------------------------
colMD                | map<int,podio::Generi... | AsGroup(<TBranchElement 'co...
colMD/colMD.first    | int32_t[]                | AsJagged(AsDtype('>i4'))
colMD/colMD.secon... | std::map<std::string,... | AsObjects(AsArray(True, Fal...
colMD/colMD.secon... | std::map<std::string,... | AsObjects(AsArray(True, Fal...
colMD/colMD.secon... | std::map<std::string,... | AsObjects(AsArray(True, Fal...
